In [ ]:
from PIL import Image
import cv2
import os
import numpy as np
import zipfile
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
class_names = [ 'Triche','Non_Triche']
# Charger les images et les labels à partir du répertoire
images = []
labels = []
data_dir='DATA_BAW'
for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if image is not None:
            image = cv2.resize(image, (200,200))  # Redimensionner l'image à la taille d'entrée du modèle
            images.append(image)
            labels.append(class_names.index(class_name))

# Convertir les listes en tableaux numpy
images = np.array(images)
labels = np.array(labels)

# Normaliser les valeurs des pixels entre 0 et 1
images = images / 255.0

# Convertir les labels en vecteurs catégoriques
labels = to_categorical(labels, len(class_names))

# Diviser les données en ensembles d'entraînement, de validation et de test


train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images,train_labels, test_size=0.2, random_state=42)
# Augmentation de données
datagen = ImageDataGenerator(rotation_range=0, zoom_range=0.3, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=False)

In [ ]:
from google.colab import drive
drive.mount('/content/https://drive.google.com/drive/folders/10GsZXRSeb4V8g1ThD5lLAufuJyINYTEf?fbclid=IwAR2eZ3JpkD8vcCJ_SCsysrD5tyD4LvIdW74TOKdFqZeHJZ8kBikixZBDIqQ')

In [ ]:
# Création du modèle CNN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(200,200, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(class_names), activation='softmax'))

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import tensorflow as tf
# Compilation du modèle
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(train_images, train_labels, epochs=6, validation_data=(val_images, val_labels))
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#model.save('model.h5')
class_names = [ 'Triche','Non_Triche']
import tensorflow as tf
model = tf.keras.models.load_model('model.h5')
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# Chemin vers le répertoire contenant les images de test sur Google Drive
test_dir = '/content/output'

# Liste des chemins d'accès des images de test
image_paths = [
    os.path.join( "Suspect", "20231031_135659.jpg"),
    os.path.join( "Suspect", "20231031_140016.jpg"),
    os.path.join( "Suspect", "20231031_141929.jpg"),
    os.path.join( "Suspect", "20231031_142444.jpg"),
    os.path.join("validation", "result1.jpg"),
    os.path.join("validation", "result18.jpg"),
    os.path.join("validation", "result24.jpg"),
    os.path.join("validation", "result20.jpg"),
    os.path.join("validation", "result23.jpg"),
    os.path.join("validation", "result5.jpg"),
    os.path.join("validation", "result16.jpg"),
    os.path.join("validation", "result17.jpg"),
    os.path.join("validation", "result1.jpg"),
    os.path.join("validation", "result14.jpg"),
    os.path.join("validation", "result13.jpg"),
    os.path.join("validation", "result26.jpg"),
    os.path.join("validation", "result2.jpg"),
    os.path.join("validation", "result19.jpg"),
    os.path.join("validation", "Capture2.jpg"),
    os.path.join("validation", "Capture3.jpg"),

]

for test_image_path in image_paths:
    # Charger l'image de test
    test_image = cv2.imread(test_image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    # Vérifier si l'image a été chargée avec succès
    if test_image is not None:
        plt.imshow(test_image, cmap='gray')
        plt.show()
        test_image = cv2.resize(test_image, (200, 200))
        # Redimensionner l'image à la taille d'entrée du modèle
        # Normaliser les valeurs des pixels entre 0 et 1
        test_image = test_image / 255.0
        # Ajouter une dimension supplémentaire pour représenter le lot
        test_image = np.expand_dims(test_image, axis=0)

        # Prédire la classe de l'image de test
        predictions = model.predict(test_image)
        probabilities = tf.nn.softmax(predictions)

        # Obtenir l'indice de classe prédit
        predicted_class_index = tf.argmax(probabilities, axis=1).numpy()[0]

        # Obtenir le nom de classe prédit
        predicted_class = class_names[predicted_class_index]

        # Afficher les résultats
        print("Image:", test_image_path)
        print("Classe prédite:", predicted_class)
        print("Probabilités:", probabilities.numpy()[0])
        print()
    else:
        print("Échec du chargement de l'image :", test_image_path)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from PIL import Image
model1 = YOLO('yolov8n-pose.pt')
video_path = 'Cas triche (exchange paper).mp4'
i = 0
counter = 0
cap = cv2.VideoCapture(video_path)
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break
  counter +=1
  if counter % 30 == 0:
    results1 = model1.predict(frame, conf=0.5, classes=[0])
    frame_color=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame=Image.fromarray(frame)
    for r in results1:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])
        for b in r.boxes.xyxy:
          b=np.array(b)
         # x=b[0]
         # y=b[1]
          #z=b[2]
          #r=b[3]
          x, y, z, r = map(int, b)
          cropped_image = frame[y:r, x:z]
          #cropped_image = frame.crop((x,y,z,r))
          cropped_image_saved=Image.fromarray(cropped_image)
          output_path = os.path.join('test', f'++result{i}++.jpg')
          cropped_image_saved.save(output_path)
          #cropped_image_new = cv2.imread(output_path)
          cropped_image = cv2.resize(cropped_image, (200, 200))
          normalized_cropped_image =cropped_image / 255.0
          normalized_cropped_image = np.expand_dims(normalized_cropped_image, axis=0)
          predictions = model.predict(normalized_cropped_image)
          probabilities = tf.nn.softmax(predictions)
          predicted_class_index = tf.argmax(probabilities, axis=1).numpy()[0]
          predicted_class = class_names[predicted_class_index]
          print(predicted_class)
          if (predicted_class=='Triche'):
            plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
            plt.show()
            plt.imshow(cv2.cvtColor(frame_color, cv2.COLOR_BGR2RGB))
            plt.show()
            print(i)
          i=i+1
    if cv2.waitKey(25) & 0xFF == ord('e'):
      break
cap.release()
cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO
import cv2
import math
from PIL import Image
import numpy as np
import os
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
model_yolo = YOLO('yolov8n-pose.pt')
counter=0
i=0
while True:

    success, img = cap.read()
    results = model_yolo(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
    cv2.imshow('Examen', img)
    if counter % 10 == 0:
        results = model_yolo(img, stream=True)
        img_color=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_NB=cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        for r in results:
           im_array = r.plot()
           im = Image.fromarray(im_array[..., ::-1])
           for b in r.boxes.xyxy:
             b=np.array(b)
             x, y, z, r = map(int, b)
             cropped_image = img_NB[y:r, x:z]
             #cropped_image_saved=Image.fromarray(cropped_image)
             #output_path = os.path.join('test', f'++result{i}++.jpg')
             #cropped_image_saved.save(output_path)
             cropped_image = cv2.resize(cropped_image, (200, 200))
             normalized_cropped_image =cropped_image / 255.0
             normalized_cropped_image = np.expand_dims(normalized_cropped_image, axis=0)
             predictions = model.predict(normalized_cropped_image)
             probabilities = tf.nn.softmax(predictions)
             predicted_class_index = tf.argmax(probabilities, axis=1).numpy()[0]
             predicted_class = class_names[predicted_class_index]
             print(predicted_class)
             if (predicted_class=='Triche'):
                cropped_image_saved = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                dossier=os.path.join('Live', f'frame{i}.jpg')
                if not os.path.exists(dossier):
                        os.makedirs(dossier)
                output_path_pil = os.path.join(dossier, f'result_pil_{counter}_{i}.jpg')
                cropped_image_saved.save(output_path_pil)
                cropped_image_saved.close()
                plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_GRAY2RGB))
                plt.show()
                img_saved = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                output_path_pil = os.path.join(dossier, f'result_{counter}_{i}.jpg')
                img_saved.save(output_path_pil)
                cropped_image_saved.close()
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.show()


    i=i+1
    counter=counter+1
    if cv2.waitKey(1) == ord('e'):
        break

cap.release()
cv2.destroyAllWindows()